# MD Package

## 1.  Load Transactional Data, SalesForce Data, Mavenlink Data, load packages, and convert to Dataframes

I am using three reports from Netsuite to populate this data:  Lists/Employees --> Run report without filters all set to "All".

Search in Netsuite search bar usings "sea" for Adaptive Transaction Search - P&L - Jeremy

Search in Netsuite search bar using "sea" for Mavenlink Time Audit

In Salesforce, use Ginny's link, but filter for "ALL" in Opportunity Record Type.  Also include Revenue Date:  greater or equal to LAST FISCAL YEAR.

![title](img/picture.jpg)

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


In [2]:
HistoricDatapath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\FY19&FY20_Detail_Trans.xlsx'

dfHistoric = pd.read_excel(HistoricDatapath)

In [3]:
FY18Datapath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\ADAPTIVETransactionSearchPLResultsFY18.xlsx'

dfFY18 = pd.read_excel(FY18Datapath)

In [4]:
GrantBalancespath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\Grant_Balance_Report_June_LOAD.xlsx'

dfGrantBalances = pd.read_excel(GrantBalancespath)

In [5]:
#Get this from Mavenlink, Insight--> Sharepoint Reports, Project List for MDs:  Export to XLSX, uncheck "Keep attribute cells merged" and "Include active filters in each report"


MavenlinkProjectspath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\Sharepoint Reports 7-20-2021 0716am.xlsx'

dfMAVELINKACTIVEPROJECTS = pd.read_excel(MavenlinkProjectspath)

In [6]:
dfHistoric = pd.concat([dfFY18, dfHistoric], ignore_index=True)

In [7]:

AdaptiveTranspath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\ADAPTIVETransactionSearchPLJeremyResults601.xls'

from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse(AdaptiveTranspath, excelHandler)
dfFY21 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

len(dfFY21)

147801

In [8]:
#Search "Mavenlink Time Audit", filter it to start at the beginning of the fiscal year.

#excelHandler = ExcelHandler()
#parse('../Data/raw/MavenlinkTimeAuditResults905.xls', excelHandler)
#dfMavenlink = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [9]:
#excelHandler = ExcelHandler()
#parse('../Data/raw/Employees588.xls', excelHandler)
#employee_list = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])


In [10]:
#https://rockymountaininstitute.lightning.force.com/lightning/r/Report/00O6f000008AUqkEAG/view?queryScope=userFolders

SalesForcePhilanthropicpath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\report1626701559081.xls'

dfSalesforce = pd.read_html(SalesForcePhilanthropicpath)
dfSalesforce = dfSalesforce[0]

In [11]:
SalesForceFFSpath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\report1626701686444.xls'

#https://rockymountaininstitute.lightning.force.com/lightning/r/Report/00O6f000008AajLEAS/view?queryScope=userFolders
dfSalesforceFFS = pd.read_html(SalesForceFFSpath)
dfSalesforceFFS = dfSalesforceFFS[0]

In [12]:
#https://rockymountaininstitute.lightning.force.com/lightning/r/Report/00O6f000008IFHbEAO/view?queryScope=userFolders

SalesForceLinksPath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\report1626701742527.xls'

dfSalesforceGrantContracts = pd.read_html(SalesForceLinksPath)
dfSalesforceGrantContracts = dfSalesforceGrantContracts[0]

In [13]:
#To get the FFS Balances and Whitney's data from Netsuite: RMI Fee for Service Contract Balances


NetsuiteContractBalancePath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\RMIFeeforServiceContractBalancesResults355.xls'

excelHandler = ExcelHandler()
parse(NetsuiteContractBalancePath, excelHandler)
dfFFSData = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [14]:
#"FFS Projects" from Netsuite
NetsuiteFFSProjectsPath =  'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\FFSProjectsResults161.xls'

excelHandler = ExcelHandler()
parse(NetsuiteFFSProjectsPath, excelHandler)
dfFFSData2 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [15]:
#To get the deferred Revenue data for FFS from Netsuite:  RMI Transaction Search - Revenue Summary

NSRMITablespath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\RMITransactionSearchRevenueSummaryResults495.xls'

excelHandler = ExcelHandler()
parse(NSRMITablespath, excelHandler)
dfFFSIncludingDeferred = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [16]:
df1 = pd.concat([dfHistoric, dfFY21], ignore_index=True)
len(df1)

478427

## 2. Data Manipulation

In this section I start manipulting these datasets into somethign more digestable for the end user.  The main manipulation I am doing in this seciton is pulling information out of the "Memo" field that users may want to see seperately.

In [17]:
#print(len(employee_id_list))

In [18]:
df1['Memo'] = df1['Memo'].astype('string')

In [19]:
df1.dtypes

Date                               object
Period                             object
GL Account                         object
Document Number                    object
Program                            object
Initiative                         object
Project                            object
Restriction                        object
Grant                              object
Amount                             object
Vendor Name                        object
Employee Name                      object
Memo                               string
Amount (Credit)                    object
Amount (Debit)                     object
Client                             object
Transaction Number                 object
Type                               object
Vendor Rate                        object
Vendor Contract Expiration Date    object
dtype: object

In [20]:
#all_ids = '|'.join(employee_id_list)

In [21]:
def return_first(x):
    try:
        y = x[0]
    except:
        y = ""
    return y

## 3. Salesforce Data Manipulation

In [22]:
dfSalesforce['Revenue Status'].replace('', np.nan, inplace=True)
dfSalesforce['Initiative'].replace('', np.nan, inplace=True)
dfSalesforce['Revenue Amount'].replace('', np.nan, inplace=True)
dfSalesforce['RMI Grant Name'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Term - Start Date'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Term - End Date'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Status'].replace('', np.nan, inplace=True)

In [23]:
dfSalesforce['Revenue Status'].fillna("na", inplace=True)
dfSalesforce['Initiative'].fillna("na", inplace=True)
dfSalesforce['Revenue Amount'].fillna(0, inplace=True)
dfSalesforce['RMI Grant Name'].fillna("na", inplace=True)
dfSalesforce['Grant Term - Start Date'].fillna("na", inplace=True)
dfSalesforce['Grant Term - End Date'].fillna("na", inplace=True)
dfSalesforce['Grant Status'].fillna("na", inplace=True)

In [24]:
dfSalesforceFFS['Amount'].replace('', np.nan, inplace=True)
dfSalesforceFFS['Relationship Manager'].replace('', np.nan, inplace=True)
dfSalesforceFFS['Initiative'].replace('', np.nan, inplace=True)

In [25]:
dfSalesforceFFS['Amount'].fillna(0, inplace=True)
dfSalesforceFFS['Relationship Manager'].fillna('na', inplace=True)
dfSalesforceFFS['Initiative'].fillna('na', inplace=True)

In [26]:
dfSalesforceGrantContracts["Documents Link"] = dfSalesforceGrantContracts['Essential Documents Link'].str.findall(r'\<a href=(.*?)\ target').apply(return_first)


In [27]:
dfSalesforceGrantContractsNS = dfSalesforceGrantContracts[['RMI Grant Name', 'Documents Link']]

In [28]:
dfSalesforceGrantContractsNS = dfSalesforceGrantContractsNS.dropna()
dfSalesforceGrantContractsNS = dfSalesforceGrantContractsNS.rename(columns={'RMI Grant Name':'Grant'})



In [29]:

dfSalesforceGrantContractsNS = dfSalesforceGrantContractsNS.drop_duplicates(subset='Grant', keep="first")

In [30]:
dfSalesforceGrantContractsNS['Documents Link'] = dfSalesforceGrantContractsNS['Documents Link'].str.replace('"', '')

In [31]:
#dfSalesforceGrantContractsNS.to_excel('test.xlsx')

In [32]:
dfSalesforceGrantContractsSF = dfSalesforceGrantContracts[['Documents Link','Opportunity Name']]

In [33]:
dfSalesforceGrantContractsSF = dfSalesforceGrantContractsSF.drop_duplicates(subset='Opportunity Name', keep="first")

In [34]:
dfSalesforceGrantContractsSF['Documents Link'] = dfSalesforceGrantContractsSF['Documents Link'].str.replace('"', '')

In [35]:
dfSalesforceFFS['Relationship Manager'].fillna('na', inplace=True)

In [36]:
len(dfSalesforce)

5947

In [37]:
dfSalesforce = pd.merge(dfSalesforce, dfSalesforceGrantContractsSF, on = 'Opportunity Name', how='left')

In [38]:
len(dfSalesforce)

5947

In [39]:
dfSalesforce['Documents Link'].fillna('na', inplace=True)

In [40]:
dfSalesforce.to_excel('Output_Salesforce.xlsx')

## 4. Manipulate Netsuite Data

In [41]:
allocations = ['BONUS ALLOCATION', 'ADMIN ALLOCATION', 'PTO/HOLIDAY ALLOCATION']

In [42]:
all_allocations = '|'.join(allocations)

In [43]:
df1['ALLOCATION'] = df1['Memo'].str.findall(all_allocations).apply(return_first)

## Extract Project Fields for Heather's report

In [44]:
df1['Project Allocations'] = df1['Project'].str.findall(r'\[(.*?)\]').apply(return_first)

## Add Rollup Accounts

In [45]:
LookupsPath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\lookups.xlsx'

lookups = pd.read_excel(LookupsPath)

In [46]:
lookups.head()

,GL Account,Rollup Account
0,5010 Contributions Revenue : Foundation Contri...,5000 Contributions Revenue
1,5015 Contributions Revenue : Campaign Foundati...,5000 Contributions Revenue
2,5020 Contributions Revenue : Individual Contri...,5000 Contributions Revenue
3,5025 Contributions Revenue : Campaign Corporat...,5000 Contributions Revenue
4,5030 Contributions Revenue : Endowment Contrib...,5000 Contributions Revenue


In [47]:
df1 = pd.merge(df1, lookups, on = 'GL Account', how='left')

In [48]:
df1['Account Digit'] = df1['GL Account'].astype('string').str[0]

In [49]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit'],
      dtype='object')

In [50]:
df1['Amount'] = df1['Amount'].astype(float)
df1['Amount (Credit)'].replace('', 0, inplace=True)
df1['Amount (Debit)'].replace('', 0, inplace=True)
df1['Amount (Credit)'] = df1['Amount (Credit)'].astype(float)
df1['Amount (Debit)'] = df1['Amount (Debit)'].astype(float)

In [51]:
df1.dtypes

Date                                object
Period                              object
GL Account                          object
Document Number                     object
Program                             object
Initiative                          object
Project                             object
Restriction                         object
Grant                               object
Amount                             float64
Vendor Name                         object
Employee Name                       object
Memo                                string
Amount (Credit)                    float64
Amount (Debit)                     float64
Client                              object
Transaction Number                  object
Type                                object
Vendor Rate                         object
Vendor Contract Expiration Date     object
ALLOCATION                          object
Project Allocations                 object
Rollup Account                      object
Account Dig

## Add Reclass Field and Closeout field

In [52]:
account_exceptions = ['To reclass', 'Reclass']

In [53]:
Accounting_Moves = '|'.join(account_exceptions)

In [54]:
closeout = 'close out'

In [55]:
df1['Accounting Adjustments'] = df1['Memo'].str.findall(Accounting_Moves).apply(return_first)

In [56]:
df1['Accounting Close Out'] = df1['Memo'].str.findall(closeout).apply(return_first)

In [57]:
df1['Vendor Contract Expiration Date'] = pd.to_datetime(df1['Vendor Contract Expiration Date'])

In [58]:
df1['Vendor Contract Expiration Date'] = df1['Vendor Contract Expiration Date'].dt.date

In [59]:
#df1 = pd.merge(df1, employeelisttomerge, on = 'Name', how='left')

In [60]:
df1['Initiative'].replace('', np.nan, inplace=True)
df1['Project'].replace('', np.nan, inplace=True)
df1['Grant'].replace('', np.nan, inplace=True)
df1['GL Account'].replace('', np.nan, inplace=True)
df1['Memo'].replace('', np.nan, inplace=True)
df1['Transaction Number'].replace('', np.nan, inplace=True)
df1['Vendor Name'].replace('', np.nan, inplace=True)
df1['Vendor Rate'].replace('', np.nan, inplace=True)
df1['Vendor Contract Expiration Date'].replace('', np.nan, inplace=True)
df1['Program'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('e', np.nan, inplace=True)
df1['Accounting Adjustments'].replace('To reclass', 'Reclass', inplace=True)
df1['Client'].replace('', np.nan, inplace=True)
df1['Rollup Account'].replace('', np.nan, inplace=True)
df1['Restriction'].replace('', np.nan, inplace=True)

C:\Users\jerem\anaconda3\lib\site-packages\pandas\core\missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [61]:
df1['Initiative'].fillna("na", inplace=True)
df1['Project'].fillna("na", inplace=True)
df1['Grant'].fillna("No Grant", inplace=True)
df1['GL Account'].fillna("na", inplace=True)
df1['Memo'].fillna("na", inplace=True)
df1['Transaction Number'].fillna("na", inplace=True)
df1['Vendor Name'].fillna("na", inplace=True)
df1['Vendor Rate'].fillna("na", inplace=True)
df1['Vendor Contract Expiration Date'].fillna("na", inplace=True)
df1['Program'].fillna("na", inplace=True)
df1['Project Allocations'].fillna("Other", inplace=True)
df1['Client'].fillna("na", inplace=True)
df1['Rollup Account'].replace("New Account", np.nan, inplace=True)
df1['Restriction'].replace("na", np.nan, inplace=True)

## Add Further Heather Calcs

In [62]:
df1['Inst or Reclass'] = df1['Account Digit'] + df1['Accounting Adjustments']

In [63]:
df1['Inst or Reclass'].value_counts()

7           368300
8            60827
5            30373
6            11336
7Reclass      5943
8Reclass      1313
5Reclass       308
6Reclass        23
9                2
Name: Inst or Reclass, dtype: Int64

In [64]:
df1['Inst or Reclass'] = df1['Inst or Reclass'].map({'7': "Expense", '8': "Institutional Overhead", '5': "Revenue", '6': "Revenue", '7Reclass': "Expense Reclass", '8Reclass': "Inst Overhead Reclass", '5Reclass': "Revenue Reclass", '6Reclass': "Revenue Reclass"})

## Unrestricted Columns Added

In [65]:
def return_grant(x):
    if x != "No Grant":
        return "Yes"
    else: 
        return "No"
    
    

In [66]:
df1['grant?'] = df1['Grant'].apply(return_grant)

In [67]:
df1["Restricted Revenue"] = df1['grant?'] + df1['Account Digit']

In [68]:
df1["Restricted Revenue"] = df1["Restricted Revenue"].map({'No7': "No", 'Yes7': "Restricted Revenue", 'No8': "No", 'No5': "No", 'Yes8': "Restricted Revenue", 'Yes6': "No", 'No6': "No", 'Yes5': "No"})

In [69]:
df1['Restricted Revenue'] = np.where(df1['Restricted Revenue'] == "Restricted Revenue", df1['Amount'], 0)

In [70]:
df1["Rev or Exp"] = df1["Account Digit"].map({'6': 'Revenue', '5': 'Revenue', '7': 'Expense', '8': 'Expense'})

In [71]:
df1["Rev or Exp"].value_counts()

Expense    436383
Revenue     42040
Name: Rev or Exp, dtype: int64

In [72]:
df1['Multilateral'] = np.where(df1['Rev or Exp'] == "Revenue", df1['Amount'], 0)

In [73]:
df1['General Expense'] = np.where(df1['Rev or Exp'] == "Expense", df1['Amount'], 0)

In [74]:
df1['Unrestricted'] = - df1['Restricted Revenue'] - df1['Multilateral'] + df1['General Expense']

In [75]:
#df1 = df1.drop('Match ID Names',1)

In [76]:
len(df1)

478427

## Build FFS Tables

In [77]:
df1['Project3YrRev'] = df1.groupby(['Project','Rev or Exp'])['Amount'].transform('sum')

dfrevenuetable = df1[['Project', 'Project3YrRev','Rev or Exp']]

#dfrevenuetable = dfrevenuetable[dfrevenuetable['Rev or Exp']=='Revenue']


In [78]:
initiative_program_table = dfMAVELINKACTIVEPROJECTS[['Program', 'Initiative', 'Project', 'Client']]

initiative_program_table['Project-edited'] = initiative_program_table['Client'] + " : " + initiative_program_table['Project']

initiative_program_table = initiative_program_table.drop_duplicates()

initiative_program_table = initiative_program_table[['Program', 'Initiative', 'Project-edited']]

initiative_program_table = initiative_program_table.rename(columns={'Project-edited':'Project'})

<ipython-input-78-5491e83c7e5f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initiative_program_table['Project-edited'] = initiative_program_table['Client'] + " : " + initiative_program_table['Project']


In [79]:
initiative_program_table2 = df1[['Program', 'Initiative', 'Project']]

In [80]:
initiative_program_table = pd.concat([initiative_program_table, initiative_program_table2], ignore_index=True)

initiative_program_table = initiative_program_table.drop_duplicates()

In [81]:
initiative_program_table

,Program,Initiative,Project
0,Industry,INY - Aviation,WSP : SFO SAF Feasibility
1,India,IN - India,WSP : Microsoft ZE Fleet Initiative
2,China,CHN - Early Peaking Cities,World Wide Fund for Nature Beijing Office - US...
3,Beijing,BEJ - Early Peaking Cities,World Wide Fund for Nature Beijing Office - CN...
4,Operations,OPS - CEO Special Projects,World Resources Institute (WRI) : America's Pl...
...,...,...,...
478777,WattTime,WAT - Admin,Toyota Motor North America WT
479052,WattTime,WAT - Admin,Invenergy Renewables LLC WT
479128,WattTime,WAT - Admin,"Facebook, Inc. WT"
479213,WattTime,WAT - Admin,Washington University WT


In [82]:
dfFFSData = dfFFSData.rename(columns={'Name':'Project'})
dfFFSIncludingDeferred = dfFFSIncludingDeferred.rename(columns={'Sum of Total Revenue Recognized':'Total Revenue Recognized'})
dfFFSData2 = dfFFSData2.rename(columns={'Current Month Invoicing Action':'Current Month Action'})
dfFFSData2 = dfFFSData2.rename(columns={'Name':'Project'})

In [83]:
dfFFSData2['DUPLICATE'] = dfFFSData2['Project'].isin(dfFFSData['Project'])

In [84]:
dfFFSData2 = dfFFSData2[dfFFSData2['DUPLICATE']==False]

In [85]:
dfFFSData2 = dfFFSData2[['Project', 'Project Type', 'Contract?', 'Contract Start Date',
       'Contract End Date', 'Contract Amount', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Project Complete?']]

In [86]:
dfFFSData = pd.concat([dfFFSData, dfFFSData2], ignore_index=True)

In [87]:
dfFFSData = pd.merge(dfFFSData, dfFFSIncludingDeferred, on = 'Project', how='left')


In [88]:
dfFFSData['Total Revenue Recognized'].replace('', np.nan, inplace=True)
dfFFSData['Total Revenue Recognized'].fillna(0, inplace=True)
dfFFSData['Reimbursables NTE'].replace('', np.nan, inplace=True)
dfFFSData['Reimbursables NTE'].fillna(0, inplace=True)
dfFFSData['Invoice Type'].replace('', np.nan, inplace=True)
dfFFSData['Invoice Type'].fillna(0, inplace=True)
dfFFSData['Invoicing Notes'].replace('', np.nan, inplace=True)
dfFFSData['Invoicing Notes'].fillna(0, inplace=True)
dfFFSData['Current Month Action'].replace('', np.nan, inplace=True)
dfFFSData['Current Month Action'].fillna(0, inplace=True)
dfFFSData['Contract Amount'].replace('', np.nan, inplace=True)
dfFFSData['Contract Amount'].fillna(0, inplace=True)
dfFFSData['Remaining Contract Balance'].replace('', np.nan, inplace=True)
dfFFSData['Remaining Contract Balance'].fillna(0, inplace=True)

In [89]:
dfFFSData['Contract Amount'] = dfFFSData['Contract Amount'].astype(float)
dfFFSData['Total Revenue Recognized'] = dfFFSData['Total Revenue Recognized'].astype(float)
dfFFSData['Reimbursables NTE'] = dfFFSData['Reimbursables NTE'].astype(float)
dfFFSData['Reimbursables Actual'] = dfFFSData['Reimbursables Actual'].astype(float)

In [90]:
dfFFSData['Remaining Revenue'] = dfFFSData['Contract Amount'] + dfFFSData['Reimbursables NTE'] - dfFFSData['Total Revenue Recognized']

In [91]:
dfFFSData.loc[dfFFSData['Remaining Revenue'] <= 0, 'Remaining Revenue'] = "na"
dfFFSData.loc[dfFFSData['Contract Amount'] <= 0, 'Contract Amount'] = "na"

In [92]:
dfFFSData['Contract Start Date'] = pd.to_datetime(dfFFSData['Contract Start Date']).dt.strftime("%m/%d/%Y")
dfFFSData['Contract End Date'] = pd.to_datetime(dfFFSData['Contract End Date']).dt.strftime("%m/%d/%Y")

In [93]:
dfFFSDataV2 = dfFFSData

dfFFSDataV2 = pd.merge(dfFFSDataV2, initiative_program_table, on = 'Project', how='left')

dfFFSDataV2 = pd.merge(dfFFSDataV2, dfrevenuetable, on = 'Project', how='left')

dfFFSDataV2 = dfFFSDataV2.drop_duplicates()

In [94]:
dfFFSDataV2 = dfFFSDataV2[dfFFSDataV2['Project'] != "Overall Total"]

In [95]:
dfFFSDataV2['Rev or Exp'].replace('', np.nan, inplace=True)
dfFFSDataV2['Rev or Exp'].fillna(0, inplace=True)
dfFFSDataV2['Reimbursables NTE'].replace('', np.nan, inplace=True)
dfFFSDataV2['Reimbursables NTE'].fillna('na', inplace=True)
dfFFSDataV2['Contract Start Date'].replace('', np.nan, inplace=True)
dfFFSDataV2['Contract Start Date'].fillna('na', inplace=True)
dfFFSDataV2['Contract End Date'].replace('', np.nan, inplace=True)
dfFFSDataV2['Contract End Date'].fillna('na', inplace=True)

In [96]:
PROJECTSWITHOUTINITIATIVES = dfFFSDataV2[pd.isnull(dfFFSDataV2['Initiative'])]

In [97]:
PROJECTSWITHOUTINITIATIVES

,Invoice Type,Main RMI Contact,Client,Project,Project Type,Contract?,Contract Start Date,Contract End Date,Invoicing Notes,Current Month Action,...,Reimbursables NTE,Reimbursables Actual,Remaining Contract Balance,Project Complete?,Total Revenue Recognized,Remaining Revenue,Program,Initiative,Project3YrRev,Rev or Exp
30842,Overall Total,,,,,,na,na,0,0,...,0.0,1929736.52,0,NaN,0.0,na,NaN,NaN,NaN,0


In [98]:
dfFFSDataV2.to_excel("FFSOpen.xlsx")

In [99]:
dfFFSDataZeros = dfFFSDataV2[dfFFSDataV2['Rev or Exp']==0]

In [100]:
#dfFFSDataZeros.to_excel('dfzeros.xlsx')

## Add critical datapoints to Netsuite Report from RMI Fee for Service Report 

In [101]:
dfFFSDataedited = dfFFSData[['Project', 'Project Type', 'Contract?',
       'Contract Start Date', 'Contract End Date', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes',
       'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE',
       'Reimbursables Actual', 'Remaining Revenue']]


dfFFSDataedited.columns

Index(['Project', 'Project Type', 'Contract?', 'Contract Start Date',
       'Contract End Date', 'Invoice Type', 'Invoicing Notes',
       'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount',
       'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual',
       'Remaining Revenue'],
      dtype='object')

In [102]:
df1 = pd.merge(df1, dfFFSDataedited, on = 'Project', how='left')

In [103]:
len(df1)

478427

In [104]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit', 'Accounting Adjustments',
       'Accounting Close Out', 'Inst or Reclass', 'grant?',
       'Restricted Revenue', 'Rev or Exp', 'Multilateral', 'General Expense',
       'Unrestricted', 'Project3YrRev', 'Project Type', 'Contract?',
       'Contract Start Date', 'Contract End Date', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes',
       'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE',
       'Reimbursables Actual', 'Remaining Revenue'],
      dtype='object')

In [105]:
df1['Project Type'].replace('', np.nan, inplace=True)
df1['Contract?'].replace('', np.nan, inplace=True)
df1['Contract Start Date'].replace('', np.nan, inplace=True)
df1['Contract End Date'].replace('', np.nan, inplace=True)
df1['Invoice Type'].replace('', np.nan, inplace=True)
df1['Invoicing Notes'].replace('', np.nan, inplace=True)
df1['Current Month Action'].replace('', np.nan, inplace=True)
df1['Revenue Recognition Notes'].replace('', np.nan, inplace=True)
df1['Contract Amount'].replace('', np.nan, inplace=True)
df1['Total Revenue Recognized'].replace('', np.nan, inplace=True)
df1['Reimbursables NTE'].replace('', np.nan, inplace=True)
df1['Reimbursables Actual'].replace('', inplace=True)
df1['Remaining Revenue'].replace('', np.nan, inplace=True)

In [106]:
df1['Project Type'].fillna("na", inplace=True)
df1['Contract?'].fillna("na", inplace=True)
df1['Contract Start Date'].fillna("na", inplace=True)
df1['Contract End Date'].fillna("na", inplace=True)
df1['Invoice Type'].fillna("na", inplace=True)
df1['Invoicing Notes'].fillna("na", inplace=True)
df1['Current Month Action'].fillna("na", inplace=True)
df1['Revenue Recognition Notes'].fillna("na", inplace=True)
df1['Contract Amount'].fillna("na", inplace=True)
df1['Total Revenue Recognized'].fillna("na", inplace=True)
df1['Reimbursables NTE'].fillna("na", inplace=True)
df1['Reimbursables Actual'].fillna("na", inplace=True)
df1['Remaining Revenue'].fillna("na", inplace=True)

## Grant Balance Data Manipulation

In [107]:
dfGrantBalances = dfGrantBalances[dfGrantBalances['Grant'].notna()]

In [108]:
dfGrantBalancescopy = dfGrantBalances[['Grant', 'Amount Received', 'Total Grant Spend', 'Institutional Overhead Note', 'Accounting Treatment', 'Institutional Overhead', 'Grant Balance', 'Grant Close Date (Adaptive Sheet)', 'Grant Start Date', 'Grant Revenue Date', 'Close Date Notes']]

In [109]:
dfGrantBalancescopy['Grant Start Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Start Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancescopy['Grant Revenue Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Revenue Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)'] = pd.to_datetime(dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)']).dt.strftime("%m/%d/%Y")

<ipython-input-109-13c56aec49e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Grant Start Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Start Date']).dt.strftime("%m/%d/%Y")
<ipython-input-109-13c56aec49e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Grant Revenue Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Revenue Date']).dt.strftime("%m/%d/%Y")
<ipython-input-109-13c56aec49e9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [110]:
dfGrantBalancescopy['Amount Received'] = dfGrantBalancescopy['Amount Received'].astype(float)
dfGrantBalancescopy['Total Grant Spend'] = dfGrantBalancescopy['Total Grant Spend'].astype(float)
dfGrantBalancescopy['Institutional Overhead'] = dfGrantBalancescopy['Institutional Overhead'].astype(float)
dfGrantBalancescopy['Grant Balance'] = dfGrantBalancescopy['Grant Balance'].astype(float)
#dfGrantBalancescopy['Close Date Notes'] = dfGrantBalancescopy['Close Date Notes'].astype(str)

<ipython-input-110-58b9aca68eff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Amount Received'] = dfGrantBalancescopy['Amount Received'].astype(float)
<ipython-input-110-58b9aca68eff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Total Grant Spend'] = dfGrantBalancescopy['Total Grant Spend'].astype(float)
<ipython-input-110-58b9aca68eff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [111]:
dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Grant Start Date'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Grant Revenue Date'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Close Date Notes'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)'].fillna("na", inplace=True)
dfGrantBalancescopy['Grant Start Date'].fillna("na", inplace=True)
dfGrantBalancescopy['Grant Revenue Date'].fillna("na", inplace=True)
dfGrantBalancescopy['Close Date Notes'].fillna("na", inplace=True)

C:\Users\jerem\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
C:\Users\jerem\anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [112]:
df1 = pd.merge(df1, dfGrantBalancescopy, on = 'Grant', how='left')

In [113]:
len(df1)

478427

# Add Hyperlinks

In [114]:
df1 = pd.merge(df1, dfSalesforceGrantContractsNS, on = 'Grant', how='left')

In [115]:
len(df1)

478427

In [116]:
df1['Grant Close Date (Adaptive Sheet)'].replace('', np.nan, inplace=True)
df1['Grant Start Date'].replace('', np.nan, inplace=True)
df1['Close Date Notes'].replace('', np.nan, inplace=True)
df1['Documents Link'].replace('', np.nan, inplace=True)
df1['Grant Revenue Date'].replace('', np.nan, inplace=True)
df1['Amount Received'].replace('', np.nan, inplace=True)
df1['Total Grant Spend'].replace('', np.nan, inplace=True)
df1['Institutional Overhead Note'].replace('', np.nan, inplace=True)
df1['Accounting Treatment'].replace('', np.nan, inplace=True)
df1['Institutional Overhead'].replace('', np.nan, inplace=True)
df1['Grant Balance'].replace('', np.nan, inplace=True)
df1['Grant Close Date (Adaptive Sheet)'].fillna("na", inplace=True)
df1['Grant Start Date'].fillna("na", inplace=True)
df1['Grant Revenue Date'].fillna("na", inplace=True)
df1['Amount Received'].fillna("na", inplace=True)
df1['Total Grant Spend'].fillna("na", inplace=True)
df1['Institutional Overhead Note'].fillna("na", inplace=True)
df1['Accounting Treatment'].fillna("na", inplace=True)
df1['Institutional Overhead'].fillna("na", inplace=True)
df1['Grant Balance'].fillna("na", inplace=True)
df1['Close Date Notes'].fillna("na", inplace=True)
df1['Documents Link'].fillna("na", inplace=True)

# Create Grant Spenddown of 0 Dataframe

In [117]:
#Missing Project
dfGrantBalancesZEROSPEND = dfGrantBalances[['Level', 'Grant', 'Amount Received', 'Total Grant Spend', 'Institutional Overhead Note', 'Accounting Treatment', 'Institutional Overhead', 'Grant Balance', 'Grant Close Date (Adaptive Sheet)', 'Grant Start Date', 'Grant Revenue Date', 'Close Date Notes']]

In [118]:
dfGrantBalancesZEROSPEND = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND['Total Grant Spend']==0]

In [119]:
dfGrantBalancesZEROSPEND["Project"] = ""

In [120]:
dfGrantBalancesZEROSPEND

,Level,Grant,Amount Received,Total Grant Spend,Institutional Overhead Note,Accounting Treatment,Institutional Overhead,Grant Balance,Grant Close Date (Adaptive Sheet),Grant Start Date,Grant Revenue Date,Close Date Notes,Project
0,D3 - Admin,FedEx - 3rd Derivative FY21,900000.00,0.0,20% of total cost,Revenue Recognized,180000.0,720000.00,2024-01-19,NaT,2021-01-25,NaN,
23,BLD - Building Electrification,Heising-Simons - Building Electrification FY21,150000.00,0.0,0% of Total Cost,NaN,0.0,150000.00,2021-11-20,2020-12-01,NaT,NaN,
24,GCF - Center for a Climate-Aligned Finance,Climateworks - CAI CCAF FY21,150000.00,0.0,0% of Total Cost,Revenue Recognized,0.0,150000.00,2021-10-31,2020-11-01,2020-12-22,NaN,
44,GCF - Center for a Climate-Aligned Finance,TD Bank - CCAF FY21,400000.00,0.0,20% of total grant,Revenue Recognized,80000.0,320000.00,NaT,NaT,2021-02-22,Anticipated end: FY23 Q2,
45,D3 - Admin,Engie - 3rd Derivative FY21,75000.00,0.0,0% of total cost,Revenue Recognized,0.0,75000.00,2024-02-01,NaT,2021-04-08,NaN,
46,GCF - Center for a Climate-Aligned Finance,CIBC - CAI CCAF FY21,150000.00,0.0,20% of total grant,Revenue Recognized,30000.0,120000.00,2022-01-29,NaT,2021-02-09,NaN,
55,BLD - Building Electrification,Moxie - Building Electrification FY20,15000.00,0.0,20% of total cost,NaN,3000.0,12000.00,NaT,NaT,NaT,Anticipated end 6/30/2021,
61,GCF - Center for a Climate-Aligned Finance,Vision Ridge - GCF FY20,100000.00,0.0,20% of total grant,NaN,20000.0,80000.00,NaT,NaT,NaT,Anticipated end 6/30/2021,
62,GCF - Center for a Climate-Aligned Finance,JPMC - CAI CCAF FY21,150000.00,0.0,20% of total grant,NaN,30000.0,120000.00,NaT,NaT,NaT,no grant agreement,
63,GCF - Center for a Climate-Aligned Finance,Goldman Sachs - CAI CCAF FY21,100000.00,0.0,20% of total cost,NaN,20000.0,80000.00,NaT,NaT,NaT,no grant agreement,


## Manipulate Dataset for BKT Changes

In [121]:
df1.loc[df1['Initiative'] == 'BKT - Market Affiliates', 'Program'] = 'Strategic Analysis and Engagement'  
df1.loc[df1['Initiative'] == 'BKT - Third Derivative', 'Program'] = 'Third Derivative'  

In [122]:
df1.loc[df1['Initiative'] == 'BKT - Market Affiliates', 'Initiative'] = 'SAE - Market Affiliates'  
df1.loc[df1['Initiative'] == 'BKT - Third Derivative', 'Initiative'] = 'D3 - Third Derivative' 
df1.loc[df1['Program'] == 'Global Climate Finance', 'Program'] = 'Finance'

In [123]:
df1.loc[(df1['Program'] == 'na') & (df1['Rollup Account'] != '7100 Employee Benefits'), 'Initiative'] = 'CH - Admin'  
df1.loc[(df1['Program'] == 'na') & (df1['Rollup Account'] != '7100 Employee Benefits'), 'Program'] = 'China'  

In [124]:
df1 = df1[df1['Date'] != 'Overall Total']

## Output manipulated DF to Excel

In [125]:
%%time
df1.columns
df1[['Date', 'Period', 'GL Account', 'Document Number', 'Program','Initiative', 'Project', 'Restriction', 'Grant', 'Amount','Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)','Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate','Vendor Contract Expiration Date', 'ALLOCATION', 'Project Allocations','Rollup Account', 'Account Digit', 'Accounting Adjustments','Accounting Close Out', 'Inst or Reclass', 'grant?','Restricted Revenue', 'Rev or Exp', 'Multilateral', 'General Expense','Unrestricted']].to_excel('Output_Netsuite.xlsx', index=False)



Wall time: 3min 11s


# 5.  Data Segmentation by Program

Here I'm taking the manipulated data and ultimately creating a dataframe to filter out each program's restricted revenue.

In [126]:
Programs = list(df1['Program'].unique())

In [127]:
print(Programs)

['Development', 'Finance', 'Operations', 'Strategic Analysis and Engagement', 'Beijing', 'China', 'Islands', 'Africa', 'Methane', 'Carbon-Free Buildings', 'Sustainable Finance', 'Sunshine for Mines', 'India', 'Carbon-Free Electricity', 'Shine', 'Trucking', 'Carbon-Free Mobility', 'WattTime', 'Shipping', 'Business Renewable Center', 'Aviation', 'Industry', 'Building Electrification', 'Communications', 'Carbon War Room', 'Emerging Energy Solutions', 'Energy Web Foundation', 'Urban Transformation', 'M3', 'na', 'Third Derivative', 'Climate Aligned Industries', 'Climate Intelligence', 'SE Asia', 'US', 'CFAN', 'Energy Transition Academy', 'Cross-Program Impact Acceleration', 'Canary Media Program']


In [128]:
def create_program(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant'] != 'No Grant'] 
    thirddf = seconddf[(seconddf['Account Digit']=='7')|(seconddf['Account Digit']=='8')]
    fourthdf = thirddf[thirddf['Restriction']=="1-Unrestricted"]
    if fourthdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return fourthdf

In [129]:
Development = create_program('Development')
Islands = create_program('Islands')
India = create_program('India')
Communications = create_program('Communications')
CrossProgramImpactAcceleration = create_program('Cross-Program Impact Acceleration')
Operations = create_program('Operations')
Development = create_program('Development')
China = create_program('China')
Breakthrough_Technology = create_program('M3')
CarbonFree_Mobility = create_program('Carbon-Free Mobility')
Climate_Aligned_Industries = create_program('Climate Aligned Industries')
Global_Climate_Finance = create_program('Finance')
Strategic_Engagement_and_Analysis_Group = create_program('Strategic Analysis and Engagement')
CarbonFree_Buildings = create_program('Carbon-Free Buildings')
WattTime = create_program('WattTime')
CarbonFree_Electricty = create_program('Carbon-Free Electricity')
Urban_Transformation = create_program('Urban Transformation')
Climate_Intelligence = create_program('Climate Intelligence')
Industry = create_program('Industry')
SE_Asia = create_program('SE Asia')
Africa = create_program('Africa')
US = create_program('US')
Building_Electrification = create_program('Building Electrification')
Energy_Transition_Academy = create_program('Energy Transition Academy')
Beijing = create_program('Beijing')
CFAN = create_program('CFAN')
D3 = create_program('Third Derivative')

Communications DF is EMPTY
Cross-Program Impact Acceleration DF is EMPTY


In [130]:
Developing_Economies_Rollup = pd.concat([Islands, Africa, SE_Asia, Energy_Transition_Academy, CFAN], ignore_index = True, sort=False)
CarbonFreeBuildings_Rollup = pd.concat([CarbonFree_Buildings, Building_Electrification], ignore_index = True, sort=False)
China_Rollup = pd.concat([China, Beijing], ignore_index = True, sort=False)

## 6. Create Reports Path and Manipulate File Names

I will need to come up with naming conventions that work over time for this project, so in this step I do just that.   Also I am loading the Openpyxl packages that i'm going to need for the rest of this program.

In [131]:
import os

DetailTransPath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Outputs\\Detail_Trans'
MDPackagePath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Outputs\\MD_Package'

In [132]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month-1, 1)
timestamp = "Detail_Transactions_"+datem.strftime("%b_%Y")

In [133]:
print(timestamp+".xlsx")

Detail_Transactions_Jun_2021.xlsx


In [134]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

## 7. DF Creation 

In [135]:
def create_df_allExpenses(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='7')|(firstdf['Account Digit']=='8')]
    thirddf = seconddf[seconddf['Restriction']=="1-Unrestricted"]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [136]:

DevelopmentExp = create_df_allExpenses("Development")
IslandsExp = create_df_allExpenses("Islands")
IndiaExp = create_df_allExpenses("India")
OperationsExp = create_df_allExpenses("Operations")
ChinaExp = create_df_allExpenses("China")
Breakthrough_Technology_Exp = create_df_allExpenses("M3")
CarbonFree_MobilityExp = create_df_allExpenses("Carbon-Free Mobility")
ClimateAligned_IndustriesExp = create_df_allExpenses("Climate Aligned Industries")
CommunicationsExp = create_df_allExpenses("Communications")
GlobalClimate_FinanceExp = create_df_allExpenses("Finance")
Strategic_Engagement_and_Analysis_GroupExp = create_df_allExpenses("Strategic Analysis and Engagement")
CrossProgramImpactAccelerationExp = create_df_allExpenses("Cross-Program Impact Acceleration")
Carbon_Free_BuildingsExp = create_df_allExpenses("Carbon-Free Buildings")
Watttime_Exp = create_df_allExpenses("WattTime")
Carbon_Free_ElectricityExp = create_df_allExpenses("Carbon-Free Electricity")
UrbanTrans_Exp = create_df_allExpenses("Urban Transformation")
ClimateIntelExp = create_df_allExpenses("Climate Intelligence")
IndustryExp = create_df_allExpenses("Industry")
SEAsiaExp = create_df_allExpenses("SE Asia")
AfricaExp = create_df_allExpenses("Africa")
USExp = create_df_allExpenses("US")
BuildingElectrificationExp = create_df_allExpenses("Building Electrification")
Energy_Transition_AcademyExp = create_df_allExpenses("Energy Transition Academy")
BeijingExp = create_df_allExpenses("Beijing")
CFANExp = create_df_allExpenses("CFAN")
D3Exp = create_df_allExpenses("Third Derivative")

In [137]:
Developing_Economies_RollupExp = pd.concat([IslandsExp, AfricaExp, SEAsiaExp, Energy_Transition_AcademyExp, CFANExp], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupExp = pd.concat([Carbon_Free_BuildingsExp, BuildingElectrificationExp], ignore_index = True, sort=False)
China_RollupExp = pd.concat([ChinaExp, BeijingExp], ignore_index = True, sort=False)

## Create FFS Income Statement Dataset

In [138]:
def create_df_FeeforService2(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant']=="No Grant"]
    thirddf = seconddf[seconddf['Restriction']=="1-Unrestricted"]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [139]:
DevelopmentFFSIS = create_df_FeeforService2("Development")
IslandsFFSIS = create_df_FeeforService2("Islands")
IndiaFFSIS = create_df_FeeforService2("India")
OperationsFFSIS = create_df_FeeforService2("Operations")
ChinaFFSIS = create_df_FeeforService2("China")
Breakthrough_Technology_FFSIS = create_df_FeeforService2("M3")
CarbonFree_MobilityFFSIS = create_df_FeeforService2("Carbon-Free Mobility")
ClimateAligned_IndustriesFFSIS = create_df_FeeforService2("Climate Aligned Industries")
CommunicationsFFSIS = create_df_FeeforService2("Communications")
GlobalClimate_FinanceFFSIS = create_df_FeeforService2("Finance")
Strategic_Engagement_and_Analysis_GroupFFSIS = create_df_FeeforService2("Strategic Analysis and Engagement")
CrossProgramImpactAccelerationFFSIS = create_df_FeeforService2("Cross-Program Impact Acceleration")
Carbon_Free_BuildingsFFSIS = create_df_FeeforService2("Carbon-Free Buildings")
Watttime_FFSIS = create_df_FeeforService2("WattTime")
Carbon_Free_ElectricityFFSIS = create_df_FeeforService2("Carbon-Free Electricity")
UrbanTrans_FFSIS = create_df_FeeforService2("Urban Transformation")
ClimateIntelFFSIS = create_df_FeeforService2("Climate Intelligence")
IndustryFFSIS = create_df_FeeforService2("Industry")
SEAsiaFFSIS = create_df_FeeforService2("SE Asia")
AfricaFFSIS = create_df_FeeforService2("Africa")
USFFSIS = create_df_FeeforService2("US")
BuildingElectrificationFFSIS = create_df_FeeforService2("Building Electrification")
Energy_Transition_AcademyFFSIS = create_df_FeeforService2("Energy Transition Academy")
BeijingFFSIS = create_df_FeeforService2("Beijing")
CFANFFSIS = create_df_FeeforService2("CFAN")
D3FFSIS = create_df_FeeforService2("Third Derivative")

In [140]:
Watttime_FFSIS = Watttime_FFSIS[Watttime_FFSIS['Rollup Account']!='5000 Contributions Revenue']

In [141]:
Developing_Economies_RollupFFSIS = pd.concat([IslandsFFSIS, AfricaFFSIS, SEAsiaFFSIS, Energy_Transition_AcademyFFSIS, CFANFFSIS], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupFFSIS = pd.concat([Carbon_Free_BuildingsFFSIS, BuildingElectrificationFFSIS], ignore_index = True, sort=False)
China_RollupFFSIS = pd.concat([ChinaFFSIS,BeijingFFSIS], ignore_index = True, sort=False)


# Create Reports for Heather and Jon Creyts

In [142]:
todayMD = datetime.today()
datemMD = datetime(today.year, today.month-1, 1)
timestampMD = datem.strftime("%B %Y")

In [143]:
timestampMD

'June 2021'

In [144]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

CFOpath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Outputs\\CFO_path'

CFOTemplatepath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\CFO_Cover_Page.xlsx'

In [145]:
blueFill = PatternFill(start_color='0033CCCC',
                   end_color='0033CCCC',
                   fill_type='solid')



## Create Cover Sheet CFO

In [146]:
Logopath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\Picture1.png'

# 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\notebooks\\img\\MD_Cover_Page.xlsx'

In [147]:
wbCFO = load_workbook(CFOTemplatepath)
wsCFO = wbCFO['Cover']    
img = openpyxl.drawing.image.Image(Logopath)
img.anchor = 'A1'
wsCFO.add_image(img)
wsCFO.cell(row = 14, column = 1).value = 'CFO Report ' + timestampMD
wbCFO.save(str(CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx'))

# 8. Managing Director Dashboard

In [148]:
MDpath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Outputs\\MD_Package'


Templatepath = 'C:\\Users\\jerem\\anaconda3\\envs\\Code\\RMICODING\\Data\\MD_Cover_Page.xlsx'

In [149]:

#wb = openpyxl.load_workbook(filename=Templatepath)
#ws = wb['Cover']

In [150]:
ProgramsMD = Programs

In [151]:
ProgramsMD.remove('na')
ProgramsMD.remove('Building Electrification')
ProgramsMD.remove('Islands')
ProgramsMD.remove('SE Asia')
ProgramsMD.remove('Africa')
ProgramsMD.remove('Energy Transition Academy')
ProgramsMD.remove('Sustainable Finance')
ProgramsMD.remove('Energy Web Foundation')
ProgramsMD.remove('Business Renewable Center')
ProgramsMD.remove('Emerging Energy Solutions')
ProgramsMD.remove('Shine')
ProgramsMD.remove('Carbon War Room')
ProgramsMD.remove('Methane')
ProgramsMD.remove('Sunshine for Mines')
ProgramsMD.remove('Aviation')
ProgramsMD.remove('Trucking')
ProgramsMD.remove('Shipping')
ProgramsMD.append('Global South')
ProgramsMD.remove('Beijing')

In [152]:
ProgramsMD

['Development',
 'Finance',
 'Operations',
 'Strategic Analysis and Engagement',
 'China',
 'Carbon-Free Buildings',
 'India',
 'Carbon-Free Electricity',
 'Carbon-Free Mobility',
 'WattTime',
 'Industry',
 'Communications',
 'Urban Transformation',
 'M3',
 'Third Derivative',
 'Climate Aligned Industries',
 'Climate Intelligence',
 'US',
 'CFAN',
 'Cross-Program Impact Acceleration',
 'Canary Media Program',
 'Global South']

## Create Cover Sheet

In [153]:
CoverNames = []

for i in ProgramsMD:
    CoverNames.append(i + " - " + timestampMD)

In [154]:
for i in CoverNames:    
    wb = load_workbook(Templatepath)
    ws = wb['Cover']    
    img = openpyxl.drawing.image.Image(Logopath)
    img.anchor = 'A1'
    ws.add_image(img)
    ws.cell(row = 14, column = 1).value = i
    wb.save(str(MDpath+'\\'+i+' - MD Package.xlsx'))

## Create First MD Table

In [155]:
initiative_program_table3 = initiative_program_table[['Program', 'Initiative']].drop_duplicates()

In [156]:
dfGrantBalancesZEROSPEND = dfGrantBalancesZEROSPEND.rename(columns={'Level':'Initiative'})

In [157]:
dfGrantBalancesZEROSPEND = pd.merge(dfGrantBalancesZEROSPEND, initiative_program_table3, on = 'Initiative', how='left')

In [158]:
dfGrantBalancesZEROSPEND = pd.merge(dfGrantBalancesZEROSPEND, dfSalesforceGrantContractsNS, on = 'Grant', how='left')

In [159]:
dfGrantBalancesZEROSPEND.columns

Index(['Initiative', 'Grant', 'Amount Received', 'Total Grant Spend',
       'Institutional Overhead Note', 'Accounting Treatment',
       'Institutional Overhead', 'Grant Balance',
       'Grant Close Date (Adaptive Sheet)', 'Grant Start Date',
       'Grant Revenue Date', 'Close Date Notes', 'Project', 'Program',
       'Documents Link'],
      dtype='object')

In [160]:
dfGrantBalancesZEROSPEND['Grant Start Date'] = pd.to_datetime(dfGrantBalancesZEROSPEND['Grant Start Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancesZEROSPEND['Grant Revenue Date'] = pd.to_datetime(dfGrantBalancesZEROSPEND['Grant Revenue Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)'] = pd.to_datetime(dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)']).dt.strftime("%m/%d/%Y")
dfGrantBalancesZEROSPEND['Amount Received'] = dfGrantBalancesZEROSPEND['Amount Received'].astype(float)
dfGrantBalancesZEROSPEND['Total Grant Spend'] = dfGrantBalancesZEROSPEND['Total Grant Spend'].astype(float)
dfGrantBalancesZEROSPEND['Institutional Overhead'] = dfGrantBalancesZEROSPEND['Institutional Overhead'].astype(float)
dfGrantBalancesZEROSPEND['Grant Balance'] = dfGrantBalancesZEROSPEND['Grant Balance'].astype(float)

In [161]:
dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Start Date'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Close Date Notes'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Revenue Date'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Amount Received'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Total Grant Spend'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead Note'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Accounting Treatment'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Balance'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Grant Start Date'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Grant Revenue Date'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Amount Received'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Total Grant Spend'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead Note'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Accounting Treatment'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Grant Balance'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Close Date Notes'].fillna("na", inplace=True)

In [162]:

ChinaGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="China"]
BEGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Building Electrification"]
CFEGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Carbon-Free Electricity"]
UTGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Urban Transformation"]
CAIGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Climate Aligned Industries"]
CFBGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Carbon-Free Buildings"]
BTGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="M3"]
CFMGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Carbon-Free Mobility"]
ISLANDSGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Islands"]
IndiaGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="India"]
GCFGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Finance"]
SEAGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Strategic Analysis and Engagement"]
WattTimeGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="WattTime"]
SEASIAGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="SE Asia"]
AfricaGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Africa"]
USGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="US"]
CIGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Climate Intelligence"]
ETAGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Energy Transition Academy"]
CFANGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="CFAN"]
D3GrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Third Derivative"]

DEGrantNOEXP = pd.concat([ISLANDSGrantNOEXP, SEASIAGrantNOEXP, AfricaGrantNOEXP, ETAGrantNOEXP, CFANGrantNOEXP], ignore_index = True, sort=False)
CFRRollingGrantNOEXP = pd.concat([BEGrantNOEXP, CFBGrantNOEXP], ignore_index = True, sort=False)

In [163]:
from openpyxl.utils.dataframe import dataframe_to_rows

def create_data_GrantZeroBalance(Group, str):
    table = Group[['Initiative','Project','Grant', 'Documents Link', 'Grant Revenue Date', 'Grant Start Date', 'Grant Close Date (Adaptive Sheet)', 'Close Date Notes', 'Accounting Treatment', 'Amount Received','Total Grant Spend', 'Institutional Overhead Note', 'Institutional Overhead',  'Grant Balance']]
    book = load_workbook(MDpath+str)
    sheet = book["Restricted Revenue"]
    data = table
    rows = dataframe_to_rows(table, index=False, header=True)
    for r_idx, row in enumerate(rows, sheet.max_row+2):
        for c_idx, value in enumerate(row, 1):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            sheet.cell(column=c_idx, row=r_idx).font = Font(bold=True)
            sheet.cell(column=c_idx, row=r_idx).number_format = '"$"#,##0;[Red]("$"#,##0)'
            if sheet.cell(column=4, row=r_idx).value is None or sheet.cell(column=4, row=r_idx).value == 'Documents Link':
                continue
            else:
                try:
                    sheet.cell(column=3, row=r_idx).hyperlink = sheet.cell(column=4, row=r_idx).value  
                    sheet.cell(column=3, row=r_idx).value = sheet.cell(column=3, row=r_idx).value    
                    sheet.cell(column=3, row=r_idx).style = "Hyperlink"   
                except:
                    pass    
            if sheet.cell(column=8, row=r_idx).value == "na" or sheet.cell(column=8, row=r_idx).value == 'Close Date Notes':
                continue
            else:
                try:
                    sheet.cell(column=7, row=r_idx).value = sheet.cell(column=8, row=r_idx).value    
                except:
                    pass    
    book.save(MDpath+str)

In [164]:
def create_MD_pivot1(Group):
    Month_order = ['Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018', 'Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'Documents Link', 'Grant Revenue Date', 'Grant Start Date', 'Grant Close Date (Adaptive Sheet)', 'Close Date Notes', 'Accounting Treatment', 'Amount Received','Total Grant Spend', 'Institutional Overhead Note', 'Institutional Overhead',  'Grant Balance'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY18-FY21 Total'] = table2.iloc[:,-49:-1].sum(axis=1)   
    return table2

In [165]:
def add_subtotal_MD1(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'','','','','','','','','','','','','Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','','','','','','','','','','','','','Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [166]:
def to_excel_MD1(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Restricted Revenue", startrow = 1)
    writer.save()
    writer.close()    

In [167]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.utils import get_column_letter

def edit_workbook_MD1(file):
    book = load_workbook(file)
    sheet = book["Restricted Revenue"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['D2']= "Grant Agreement Link"       
    sheet['E2']= "Grant Revenue Date"    
    sheet['F2']= "Grant Start Date"    
    sheet['G2']= "Grant End Date"  
    sheet['H2']= "Close Date Notes" 
    sheet['I2']= "Accounting Treatment "    
    sheet['J2']= "Amount Received"    
    sheet['K2']= "Total Grant Spend"    
    sheet['L2']= "Institutional Overhead Note"    
    sheet['M2']= "Institutional Overhead"    
    sheet['N2']= "Grant Balance"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
    sheet['E2'].font = Font(bold=True)
    sheet['F2'].font = Font(bold=True)
    sheet['G2'].font = Font(bold=True)
    sheet['H2'].font = Font(bold=True)
    sheet['I2'].font = Font(bold=True)
    sheet['J2'].font = Font(bold=True)
    sheet['K2'].font = Font(bold=True)
    sheet['L2'].font = Font(bold=True)
    sheet['M2'].font = Font(bold=True)
#    fontStyle = Font(size = "26")
    sheet.merge_cells('A1:BL1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['BL1']
    top_left_cell.value = "FY21 Restricted Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 35
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['BK'].width = 15
    sheet.column_dimensions['BL'].width = 15
    sheet.column_dimensions['E'].hidden = True
    sheet.column_dimensions['F'].hidden = True  
    sheet.column_dimensions['I'].hidden = True  
#    sheet.column_dimensions['BL'].hidden = True
    sheet.column_dimensions['D'].hidden = True
    sheet.column_dimensions['H'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))
    for col in range(3, 4):
        for row in range(2, num_rows+1):
            if sheet.cell(column=4, row=row).value == 'na':
                continue
            else:
                try:
                    sheet.cell(column=col, row=row).hyperlink = sheet.cell(column=4, row=row).value  
                    sheet.cell(column=col, row=row).value = sheet.cell(column=col, row=row).value    
                    sheet.cell(column=col, row=row).style = "Hyperlink"   
                except:
                    pass    
    for col in range(3, 4):
        for row in range(2, num_rows+1):
                sheet.cell(column=col, row=row).border = thin_border
    for col in range(7, 8):
        for row in range(2, num_rows+1):
            if sheet.cell(column=8, row=row).value == 'na' or sheet.cell(column=8, row=row).value == 'Close Date Notes':
                continue
            else:
                try:
                    sheet.cell(column=7, row=row).value = sheet.cell(column=8, row=row).value      
                except:
                    pass    
    for col in range(9, 65):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'            
    for col in range(63, 66):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)            
    for col in range(1, 15):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=14).value == "Subtotal":
            for j in range(4,65):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))   
            for j in range(63,65):
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'), left=thin) 
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,65):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,65):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(65, 66):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)  
#    sheet.auto_filter.ref = sheet.dimensions
    FullRange = "A2:" + get_column_letter(sheet.max_column) \
    + str(sheet.max_row)
    sheet.auto_filter.ref = FullRange
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "O3"
    sheet.column_dimensions.group("O","AX",hidden=True)
    book.save(file)

In [168]:
def format_multi_pivot_MD1(Group, str):
    step1 = create_MD_pivot1(Group)
    step2 = add_subtotal_MD1(step1)
    to_excel_MD1(step2, MDpath+str)
    edit_workbook_MD1(MDpath+str)

In [169]:
CoverNames

['Development - June 2021',
 'Finance - June 2021',
 'Operations - June 2021',
 'Strategic Analysis and Engagement - June 2021',
 'China - June 2021',
 'Carbon-Free Buildings - June 2021',
 'India - June 2021',
 'Carbon-Free Electricity - June 2021',
 'Carbon-Free Mobility - June 2021',
 'WattTime - June 2021',
 'Industry - June 2021',
 'Communications - June 2021',
 'Urban Transformation - June 2021',
 'M3 - June 2021',
 'Third Derivative - June 2021',
 'Climate Aligned Industries - June 2021',
 'Climate Intelligence - June 2021',
 'US - June 2021',
 'CFAN - June 2021',
 'Cross-Program Impact Acceleration - June 2021',
 'Canary Media Program - June 2021',
 'Global South - June 2021']

In [170]:
#!pip install openpyxl==3.0.7

In [171]:
format_multi_pivot_MD1(Developing_Economies_Rollup, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Climate_Aligned_Industries, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(India, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Operations, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1(Development, '\\Development - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1(Communications, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(China_Rollup, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Breakthrough_Technology, '\\M3 - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1(CrossProgramImpactAcceleration, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFree_Mobility, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFree_Electricty, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Strategic_Engagement_and_Analysis_Group, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFreeBuildings_Rollup, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Urban_Transformation, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Climate_Intelligence, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Industry, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(US, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(WattTime, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Global_Climate_Finance, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(D3, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')

In [172]:
create_data_GrantZeroBalance(ChinaGrantNOEXP,'\\China - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CFEGrantNOEXP,'\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(UTGrantNOEXP,'\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CAIGrantNOEXP,'\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CFRRollingGrantNOEXP,'\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(BTGrantNOEXP,'\\M3 - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CFMGrantNOEXP,'\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(IndiaGrantNOEXP,'\\India - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(GCFGrantNOEXP,'\\Finance - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(SEAGrantNOEXP,'\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(WattTimeGrantNOEXP,'\\WattTime - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(USGrantNOEXP,'\\US - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CIGrantNOEXP,'\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(DEGrantNOEXP,'\\Global South - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(D3GrantNOEXP,'\\Third Derivative - '+timestampMD+' - MD Package.xlsx')

## Restricted Revenue tab by Grant and Project

In [173]:
def create_MD_pivot1a(Group):
    Month_order = ['Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018', 'Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Grant','Project','Documents Link', 'Grant Revenue Date', 'Grant Start Date', 'Grant Close Date (Adaptive Sheet)', 'Close Date Notes', 'Accounting Treatment', 'Amount Received','Total Grant Spend', 'Institutional Overhead Note', 'Institutional Overhead',  'Grant Balance'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY18-FY21 Total'] = table2.iloc[:,-49:-1].sum(axis=1)   
    return table2

In [174]:
def add_subtotal_MD1a(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'','','','','','','','','','','','','Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','','','','','','','','','','','','','Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [175]:
def to_excel_MD1a(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Restricted Revenue ReOrg", startrow = 1)
    writer.save()
    writer.close()  

In [176]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.utils import get_column_letter

def edit_workbook_MD1a(file):
    book = load_workbook(file)
    sheet = book["Restricted Revenue ReOrg"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Grant"
    sheet['C2']= "Project"    
    sheet['D2']= "Grant Agreement Link"       
    sheet['E2']= "Grant Revenue Date"    
    sheet['F2']= "Grant Start Date"    
    sheet['G2']= "Grant Close Date"  
    sheet['H2']= "Close Date Notes" 
    sheet['I2']= "Accounting Treatment "    
    sheet['J2']= "Amount Received"    
    sheet['K2']= "Total Grant Spend"    
    sheet['L2']= "Institutional Overhead Note"    
    sheet['M2']= "Institutional Overhead"    
    sheet['N2']= "Grant Balance"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
    sheet['E2'].font = Font(bold=True)
    sheet['F2'].font = Font(bold=True)
    sheet['G2'].font = Font(bold=True)
    sheet['H2'].font = Font(bold=True)
    sheet['I2'].font = Font(bold=True)
    sheet['J2'].font = Font(bold=True)
    sheet['K2'].font = Font(bold=True)
    sheet['L2'].font = Font(bold=True)
    sheet['M2'].font = Font(bold=True)
#    fontStyle = Font(size = "26")
    sheet.merge_cells('A1:BL1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['BL1']
    top_left_cell.value = "FY21 Restricted Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 35
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['BK'].width = 15
    sheet.column_dimensions['BL'].width = 15
    sheet.column_dimensions['E'].hidden = True
    sheet.column_dimensions['F'].hidden = True  
    sheet.column_dimensions['I'].hidden = True  
#    sheet.column_dimensions['BL'].hidden = True
    sheet.column_dimensions['D'].hidden = True
    sheet.column_dimensions['H'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))
    for col in range(2, 3):
        for row in range(2, num_rows+1):
            if sheet.cell(column=4, row=row).value == 'na':
                continue
            else:
                try:
                    sheet.cell(column=col, row=row).hyperlink = sheet.cell(column=4, row=row).value  
                    sheet.cell(column=col, row=row).value = sheet.cell(column=col, row=row).value    
                    sheet.cell(column=col, row=row).style = "Hyperlink"   
                except:
                    pass    
    for col in range(3, 4):
        for row in range(2, num_rows+1):
                sheet.cell(column=col, row=row).border = thin_border
    for col in range(7, 8):
        for row in range(2, num_rows+1):
            if sheet.cell(column=8, row=row).value == 'na' or sheet.cell(column=8, row=row).value == 'Close Date Notes':
                continue
            else:
                try:
                    sheet.cell(column=7, row=row).value = sheet.cell(column=8, row=row).value      
                except:
                    pass    
    for col in range(9, 65):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'            
    for col in range(63, 66):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)            
    for col in range(1, 15):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=14).value == "Subtotal":
            for j in range(4,65):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))   
            for j in range(63,65):
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'), left=thin) 
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,65):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,65):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(65, 66):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)  
#    sheet.auto_filter.ref = sheet.dimensions
    FullRange = "A2:" + get_column_letter(sheet.max_column) \
    + str(sheet.max_row)
    sheet.auto_filter.ref = FullRange
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "O3"
    sheet.sheet_state = 'hidden'
    sheet.column_dimensions.group("O","AX",hidden=True)
    book.save(file)

In [177]:
def format_multi_pivot_MD1a(Group, str):
    step1 = create_MD_pivot1a(Group)
    step2 = add_subtotal_MD1a(step1)
    to_excel_MD1a(step2, MDpath+str)
    edit_workbook_MD1a(MDpath+str)

In [178]:
format_multi_pivot_MD1a(Developing_Economies_Rollup, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Climate_Aligned_Industries, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(India, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Operations, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1a(Development, '\\Development - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1a(Communications, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(China_Rollup, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Breakthrough_Technology, '\\M3 - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1a(CrossProgramImpactAcceleration, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(CarbonFree_Mobility, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(CarbonFree_Electricty, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Strategic_Engagement_and_Analysis_Group, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(CarbonFreeBuildings_Rollup, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Urban_Transformation, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Climate_Intelligence, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Industry, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(US, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(WattTime, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(Global_Climate_Finance, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1a(D3, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')

In [179]:
from openpyxl.utils.dataframe import dataframe_to_rows

def create_data_GrantZeroBalance2(Group, str):
    table = Group[['Initiative','Grant','Project', 'Documents Link', 'Grant Revenue Date', 'Grant Start Date', 'Grant Close Date (Adaptive Sheet)', 'Close Date Notes', 'Accounting Treatment', 'Amount Received','Total Grant Spend', 'Institutional Overhead Note', 'Institutional Overhead',  'Grant Balance']]
    book = load_workbook(MDpath+str)
    sheet = book["Restricted Revenue ReOrg"]
    data = table
    rows = dataframe_to_rows(table, index=False, header=True)
    for r_idx, row in enumerate(rows, sheet.max_row+2):
        for c_idx, value in enumerate(row, 1):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            sheet.cell(column=c_idx, row=r_idx).font = Font(bold=True)
            sheet.cell(column=c_idx, row=r_idx).number_format = '"$"#,##0;[Red]("$"#,##0)'
            if sheet.cell(column=4, row=r_idx).value is None or sheet.cell(column=4, row=r_idx).value == 'Documents Link':
                continue
            else:
                try:
                    sheet.cell(column=3, row=r_idx).hyperlink = sheet.cell(column=4, row=r_idx).value  
                    sheet.cell(column=3, row=r_idx).value = sheet.cell(column=3, row=r_idx).value    
                    sheet.cell(column=3, row=r_idx).style = "Hyperlink"   
                except:
                    pass    
            if sheet.cell(column=8, row=r_idx).value == "na" or sheet.cell(column=8, row=r_idx).value == 'Close Date Notes':
                continue
            else:
                try:
                    sheet.cell(column=7, row=r_idx).value = sheet.cell(column=8, row=r_idx).value    
                except:
                    pass    
    book.save(MDpath+str)

In [180]:
create_data_GrantZeroBalance2(ChinaGrantNOEXP,'\\China - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(CFEGrantNOEXP,'\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(UTGrantNOEXP,'\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(CAIGrantNOEXP,'\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(CFRRollingGrantNOEXP,'\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(BTGrantNOEXP,'\\M3 - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(CFMGrantNOEXP,'\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(IndiaGrantNOEXP,'\\India - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(GCFGrantNOEXP,'\\Finance - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(SEAGrantNOEXP,'\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(WattTimeGrantNOEXP,'\\WattTime - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(USGrantNOEXP,'\\US - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(CIGrantNOEXP,'\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(DEGrantNOEXP,'\\Global South - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance2(DEGrantNOEXP,'\\Third Derivative - '+timestampMD+' - MD Package.xlsx')


## Create FFS Dataset

In [181]:
dfFFSDataZeros = dfFFSDataZeros[['Program','Initiative','Project','Contract?','Project Type', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue']]

dfFFSDataZeros["Revenue/Expense"] = ""
dfFFSDataZeros["Rollup Account"] = ""

ChinaFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="China"]
CFEFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Electricity"]
UTFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Urban Transformation"]
CAIFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Climate Aligned Industries"]
CFBFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Buildings"]
BTFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="M3"]
CFMFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Mobility"]
ISLANDSFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Islands"]
CIFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Climate Intelligence"]
D3NOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Third Derivative"]


## Create Total Expenses by Initiative and Project

In [182]:
def create_MD_pivot7(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Rollup Account'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [183]:
def add_subtotal_MD7(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'','Subtotal'))) for k, d in PivotedGroup.groupby('Project')]).append((PivotedGroup.sum()).rename(('Grand', '','Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [184]:
def to_excel_MD7(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Expenses by Project", startrow = 1)
    writer.save()
    writer.close()    

In [185]:
def edit_workbook_MD7(file):
    book = load_workbook(file)
    sheet = book["Expenses by Project"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Account Category"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "FY21 Total Program Expenses by Initiative and Project"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 30
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    sheet.sheet_state = 'hidden'
    book.save(file)

In [186]:
def format_multi_pivot_MD7(Group, str):
    step1 = create_MD_pivot7(Group)
    step2 = add_subtotal_MD7(step1)
    to_excel_MD7(step2, MDpath+str)
    edit_workbook_MD7(MDpath+str)

In [187]:
format_multi_pivot_MD7(Developing_Economies_RollupExp, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(ClimateAligned_IndustriesExp, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(IndiaExp, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(OperationsExp, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(DevelopmentExp, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CommunicationsExp, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(ChinaExp, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Breakthrough_Technology_Exp, '\\M3 - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CarbonFree_MobilityExp, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Carbon_Free_ElectricityExp, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CrossProgramImpactAccelerationExp, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CarbonFreeBuildings_RollupExp, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(UrbanTrans_Exp, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(ClimateIntelExp, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(IndustryExp, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(USExp, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(GlobalClimate_FinanceExp, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Watttime_Exp, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(D3Exp, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

## Restricted Revenue Spend by Grant and Project

In [188]:
def create_MD_pivot8(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Grant','Project'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [189]:
def to_excel_MD8(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Grant by Project", startrow = 1)
    writer.save()
    writer.close()    

In [190]:
def edit_workbook_MD8(file):
    book = load_workbook(file)
    sheet = book["Grant by Project"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Grant"
    sheet['B2']= "Project"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "FY21 Total Program Expenses by Grant and Project"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 45
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grant":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    sheet.sheet_state = 'hidden'
    book.save(file)

In [191]:
def format_multi_pivot_MD8(Group, str):
    step1 = create_MD_pivot8(Group)
    to_excel_MD8(step1, MDpath+str)
    edit_workbook_MD8(MDpath+str)

In [192]:
format_multi_pivot_MD8(Developing_Economies_Rollup, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Climate_Aligned_Industries, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(India, '\\India - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD8(Operations, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD8(Development, '\\Development - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD8(Communications, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(China_Rollup, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Breakthrough_Technology, '\\M3 - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(CarbonFree_Mobility, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(CarbonFree_Electricty, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD8(CrossProgramImpactAcceleration, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Strategic_Engagement_and_Analysis_Group, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(CarbonFreeBuildings_Rollup, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Urban_Transformation, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Climate_Intelligence, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Industry, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(US, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Global_Climate_Finance, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(Watttime_Exp, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD8(D3Exp, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

## Income Statement for FFS by Project

In [193]:
def create_MD_pivot9(Group):
    Month_order = ['Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018','Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Project','Rev or Exp', 'Rollup Account','Project Type', 'Contract?', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY18-FY21 Total'] = table2.iloc[:,-49:-1].sum(axis=1)
    return table2

In [194]:
def add_subtotal_MD9(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(( k,'','','','','','','','','','','','','','','Unrestricted Used'))) for k, d in PivotedGroup.groupby('Project')]).append((PivotedGroup.sum()).rename(('Total', '','','','','','','','','','','','','','', 'UNRESTRICTED Used')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [195]:
def to_excel_MD9(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book
   
    df.to_excel(writer, sheet_name="FFS & Admin Income Statement", startrow = 1)
    writer.save()
    writer.close()    

In [196]:
def edit_workbook_MD9(file):
    book = load_workbook(file)
    sheet = book["FFS & Admin Income Statement"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Project"
    sheet['B2']= "Revenue/Expense"
    sheet['C2']= "Rollup Account"
    sheet['D2']= "Project Type"
    sheet['E2']= "Contract?"
    sheet['F2']= "Contract Start Date"
    sheet['G2']= "Contract End Date"
    sheet['H2']= "Invoice Type"
    sheet['I2']= "Invoicing Notes"
    sheet['J2']= "Current Month Action"
    sheet['K2']= "Revenue Recognition Notes"
    sheet['L2']= "Contract Amount"
    sheet['M2']= "Total Revenue Recognized (INCLUDES DEFERRED REVENUE)"
    sheet['N2']= "Reimbursables NTE"
    sheet['O2']= "Reimbursables Actual"
    sheet['P2']= "Remaining Revenue"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:BN1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['BN1']
    top_left_cell.value = "FY21 Earned, Government, and Multilateral Grant Income Statement"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 15
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 10
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 20
    sheet.column_dimensions['BM'].width = 15
    sheet.column_dimensions['BN'].width = 15
    sheet.column_dimensions['F'].hidden = True
    sheet.column_dimensions['J'].hidden = True
    sheet.column_dimensions['N'].hidden = True
    sheet.column_dimensions['O'].hidden = True
    sheet.column_dimensions['K'].hidden = True
#    sheet.column_dimensions['BB'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(1, 17):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for col in range(11, 67):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(65, 68):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=16).value == "Unrestricted Used":
            for j in range(3,67):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))    
            for j in range(65,67):       
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'),left=thin)    
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Total":
            for j in range(1,67):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Project":
            for j in range(1,67):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
#    sheet.auto_filter.ref = sheet.dimensions
    FullRange = "A2:" + get_column_letter(sheet.max_column) \
    + str(sheet.max_row)
    sheet.auto_filter.ref = FullRange
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "Q3"
    sheet.column_dimensions.group("Q","AZ",hidden=True)
    sheet.column_dimensions.group("D","O",hidden=True)
    book.save(file)

In [197]:
def format_multi_pivot_MD9(Group, str):
    step1 = create_MD_pivot9(Group)
    step2 = add_subtotal_MD9(step1)
    to_excel_MD9(step2, MDpath+str)
    edit_workbook_MD9(MDpath+str)

In [198]:

format_multi_pivot_MD9(ClimateAligned_IndustriesFFSIS, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(IndiaFFSIS, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(OperationsFFSIS, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(DevelopmentFFSIS, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(CommunicationsFFSIS, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(ChinaFFSIS, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(Breakthrough_Technology_FFSIS, '\\M3 - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(CarbonFree_MobilityFFSIS, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(Carbon_Free_ElectricityFFSIS, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(CrossProgramImpactAccelerationFFSIS, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(Strategic_Engagement_and_Analysis_GroupFFSIS, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(CarbonFreeBuildings_RollupFFSIS, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(UrbanTrans_FFSIS, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(ClimateIntelFFSIS, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(IndustryFFSIS, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(GlobalClimate_FinanceFFSIS, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(Watttime_FFSIS, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD9(D3FFSIS, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD9(USFFSIS, '\\US - '+timestampMD+".xlsx")

In [199]:
def create_data_FFSZeroBalance2(Group, str):
    table = Group[['Project','Revenue/Expense','Rollup Account','Project Type','Contract?','Contract Start Date', 'Contract End Date','Invoice Type',  'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue']]
    book = load_workbook(MDpath+str)
    sheet = book["FFS & Admin Income Statement"]
    data = table
    rows = dataframe_to_rows(table, index=False, header=True)
    for r_idx, row in enumerate(rows, sheet.max_row+2):
        for c_idx, value in enumerate(row, 1):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            sheet.cell(column=c_idx, row=r_idx).font = Font(bold=True)
            sheet.cell(column=c_idx, row=r_idx).number_format = '"$"#,##0;[Red]("$"#,##0)'
    book.save(MDpath+str)

In [200]:
create_data_FFSZeroBalance2(CFEFFSNOREV,'\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(ChinaFFSNOREV,'\\China - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(UTFFSNOREV,'\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(CAIFFSNOREV,'\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(CFBFFSNOREV,'\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(BTFFSNOREV,'\\M3 - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(CFMFFSNOREV,'\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(CIFFSNOREV,'\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance2(D3NOREV,'\\Third Derivative - '+timestampMD+' - MD Package.xlsx')



## Income Statement for FFS by Program and Project

In [201]:
def create_MD_pivot10(Group):
    Month_order = ['Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018','Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Program','Project','Rev or Exp', 'Rollup Account','Project Type', 'Contract?', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY18-FY21 Total'] = table2.iloc[:,-49:-1].sum(axis=1)
    return table2

In [202]:
def add_subtotal_MD10(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(( '', k,'','','','','','','','','','','','','','','Unrestricted Used'))) for k, d in PivotedGroup.groupby('Project')]).append((PivotedGroup.sum()).rename(('','Total', '','','','','','','','','','','','','','', 'UNRESTRICTED USED')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [203]:
def to_excel_MD10(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book
   
    df.to_excel(writer, sheet_name="FFS Income Statement", startrow = 1)
    writer.save()
    writer.close()    

In [204]:
def edit_workbook_MD10(file):
    book = load_workbook(file)
    sheet = book["FFS & Admin Income Statement"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Program"
    sheet['B2']= "Project"
    sheet['C2']= "Revenue/Expense"
    sheet['D2']= "Rollup Account"
    sheet['E2']= "Project Type"
    sheet['F2']= "Contract?"
    sheet['G2']= "Contract Start Date"
    sheet['H2']= "Contract End Date"
    sheet['I2']= "Invoice Type"
    sheet['J2']= "Invoicing Notes"
    sheet['K2']= "Current Month Action"
    sheet['L2']= "Revenue Recognition Notes"
    sheet['M2']= "Contract Amount"
    sheet['N2']= "Total Revenue Recognized (INCLUDES DEFERRED REVENUE)"
    sheet['O2']= "Reimbursables NTE"
    sheet['P2']= "Reimbursables Actual"
    sheet['Q2']= "Remaining Revenue"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:BO1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['BO1']
    top_left_cell.value = "FY21 Earned, Government, and Multilateral Grant Income Statement"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 20
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['BN'].width = 15
    sheet.column_dimensions['BO'].width = 15
    sheet.column_dimensions['G'].hidden = True
    sheet.column_dimensions['K'].hidden = True
    sheet.column_dimensions['M'].hidden = True
    sheet.column_dimensions['P'].hidden = True
    sheet.column_dimensions['L'].hidden = True
#    sheet.column_dimensions['BC'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(1, 18):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for col in range(12, 68):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(66, 69):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=17).value == "Unrestricted Used":
            for j in range(4,68):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))  
            for j in range(66,68):
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'),left=thin)  
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Total":
            for j in range(1,68):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Program":
            for j in range(1,68):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
#    sheet.auto_filter.ref = sheet.dimensions
    FullRange = "A2:" + get_column_letter(sheet.max_column) \
    + str(sheet.max_row)
    sheet.auto_filter.ref = FullRange
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "R3"
    sheet.column_dimensions.group("R","BA",hidden=True)
    sheet.column_dimensions.group("E","P",hidden=True)
    book.save(file)

In [205]:
def format_multi_pivot_MD10(Group, str):
    step1 = create_MD_pivot10(Group)
    step2 = add_subtotal_MD10(step1)
    to_excel_MD9(step2, MDpath+str)
    edit_workbook_MD10(MDpath+str)

In [206]:
format_multi_pivot_MD10(Developing_Economies_RollupFFSIS, '\\Global South - '+timestampMD+' - MD Package.xlsx')

In [207]:
create_data_FFSZeroBalance2(ISLANDSFFSNOREV,'\\Global South - '+timestampMD+' - MD Package.xlsx')

# Create All Expenses Report by Initiative and Account

In [208]:
def create_MD_pivot3(Group):
    Month_order = ['Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018','Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Rollup Account'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [209]:
def add_subtotal_MD3(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [210]:
def to_excel_MD3(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Total Expenses", startrow = 1)
    writer.save()
    writer.close()    

In [211]:
def edit_workbook_MD3(file):
    book = load_workbook(file)
    sheet = book["Total Expenses"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Account Category"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:AY1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['AY1']
    top_left_cell.value = "FY21 Total Program Expenses by Initiative and Account"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 35
    sheet.column_dimensions['AM'].width = 15
    sheet.column_dimensions['AN'].width = 15
    sheet.column_dimensions['AO'].width = 15
    sheet.column_dimensions['AP'].width = 15
    sheet.column_dimensions['AQ'].width = 15
    sheet.column_dimensions['AR'].width = 15
    sheet.column_dimensions['AS'].width = 15
    sheet.column_dimensions['AT'].width = 15
    sheet.column_dimensions['AU'].width = 15
    sheet.column_dimensions['AV'].width = 15
    sheet.column_dimensions['AW'].width = 15
    sheet.column_dimensions['AX'].width = 15
    sheet.column_dimensions['AY'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 52):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,52):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,52):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,52):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(52, 53):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    sheet.column_dimensions.group("C","AL",hidden=True)
    book.save(file)

In [212]:
def format_multi_pivot_MD3(Group, str):
    step1 = create_MD_pivot3(Group)
    step2 = add_subtotal_MD3(step1)
    to_excel_MD3(step2, MDpath+str)
    edit_workbook_MD3(MDpath+str)

In [213]:
format_multi_pivot_MD3(Developing_Economies_RollupExp, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ClimateAligned_IndustriesExp, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(IndiaExp, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(OperationsExp, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(DevelopmentExp, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CommunicationsExp, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ChinaExp, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Breakthrough_Technology_Exp, '\\M3 - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CarbonFree_MobilityExp, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CrossProgramImpactAccelerationExp, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Carbon_Free_ElectricityExp, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CarbonFreeBuildings_RollupExp, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(UrbanTrans_Exp, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ClimateIntelExp, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(IndustryExp, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(USExp, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(GlobalClimate_FinanceExp, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Watttime_Exp, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(D3Exp, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

# Unrestricted Report

In [214]:
def create_Totalprogram(program):
    firstdf = df1[df1['Program']==program]
    thirddf = firstdf[firstdf['Restriction']=="1-Unrestricted"]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf
    


In [215]:
DevelopmentTotal = create_Totalprogram('Development')
CommunicationsTotal = create_Totalprogram('Communications')
IslandsTotal = create_Totalprogram('Islands')
IndiaTotal = create_Totalprogram('India')
OperationsTotal = create_Totalprogram('Operations')
ChinaTotal = create_Totalprogram('China')
Breakthrough_TechnologyTotal = create_Totalprogram('M3')
CarbonFree_MobilityTotal = create_Totalprogram('Carbon-Free Mobility')
Climate_Aligned_IndustriesTotal = create_Totalprogram('Climate Aligned Industries')
Global_Climate_FinanceTotal = create_Totalprogram('Finance')
Strategic_Engagement_and_Analysis_GroupTotal = create_Totalprogram('Strategic Analysis and Engagement')
CrossProgramImpactAccelerationTotal = create_Totalprogram('Cross-Program Impact Acceleration')
CarbonFree_BuildingsTotal = create_Totalprogram('Carbon-Free Buildings')
WattTimeTotal = create_Totalprogram('WattTime')
CarbonFree_ElectrictyTotal = create_Totalprogram('Carbon-Free Electricity')
Urban_TransformationTotal = create_Totalprogram('Urban Transformation')
Climate_IntelligenceTotal = create_Totalprogram('Climate Intelligence')
IndustryTotal = create_Totalprogram('Industry')
SE_AsiaTotal = create_Totalprogram('SE Asia')
AfricaTotal = create_Totalprogram('Africa')
USTotal = create_Totalprogram('US')
Building_ElectrificationTotal = create_Totalprogram('Building Electrification')
Energy_Transition_AcademyTotal = create_Totalprogram('Energy Transition Academy')
BeijingTotal = create_Totalprogram('Beijing')
CFANTotal = create_Totalprogram('CFAN')
D3Total = create_Totalprogram('Third Derivative')

In [216]:
WattTimeTotal = WattTimeTotal[WattTimeTotal['Rollup Account']!='5000 Contributions Revenue']

In [217]:
Developing_Economies_RollupTotal = pd.concat([IslandsTotal, AfricaTotal, SE_AsiaTotal, Energy_Transition_AcademyTotal, CFANTotal], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupTotal = pd.concat([CarbonFree_BuildingsTotal, Building_ElectrificationTotal], ignore_index = True, sort=False)
China_RollupTotal = pd.concat([ChinaTotal, BeijingTotal], ignore_index = True, sort=False)

In [218]:
def create_MD_pivot4(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [219]:
def add_subtotal_MD4(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [220]:
def to_excel_MD4(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted Use", startrow = 1)
    writer.save()
    writer.close()    

In [221]:
def edit_workbook_MD4(file):
    book = load_workbook(file)
    sheet = book["Unrestricted Use"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "FY21 Unrestricted Use by Initiative, Project, and Grant"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 35
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin) 
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [222]:
def format_multi_pivot_MD4(Group, str):
    step1 = create_MD_pivot4(Group)
    step2 = add_subtotal_MD4(step1)
    to_excel_MD4(step2, MDpath+str)
    edit_workbook_MD4(MDpath+str)

In [223]:
format_multi_pivot_MD4(Developing_Economies_RollupTotal, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Climate_Aligned_IndustriesTotal, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(IndiaTotal, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(OperationsTotal, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(DevelopmentTotal, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CommunicationsTotal, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(China_RollupTotal, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Breakthrough_TechnologyTotal, '\\M3 - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CrossProgramImpactAccelerationTotal, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFree_MobilityTotal, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFree_ElectrictyTotal, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Strategic_Engagement_and_Analysis_GroupTotal, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFreeBuildings_RollupTotal, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Urban_TransformationTotal, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Climate_IntelligenceTotal, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(IndustryTotal, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(USTotal, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(WattTimeTotal, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Global_Climate_FinanceTotal, '\\Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(D3Total, '\\Third Derivative - '+timestampMD+' - MD Package.xlsx')


## Create Salesforce Pivots for Philanthropic

In [224]:
def create_Salesforceprogram(program):
    firstdf = dfSalesforce[dfSalesforce['Program: Program Name']==program]
    seconddf = firstdf[firstdf['Grant Status']!='Complete - Satisfactory']
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [225]:
DevelopmentSF = create_Salesforceprogram("General Support")
IslandsSF = create_Salesforceprogram("Islands")
IndiaSF = create_Salesforceprogram("India")
OperationsSF = create_Salesforceprogram("Operations")
ChinaSF = create_Salesforceprogram("China")
Breakthrough_Technology_SF = create_Salesforceprogram("Breakthrough Technology")
CarbonFree_MobilitySF = create_Salesforceprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesSF = create_Salesforceprogram("Climate Aligned Industries")
CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSF = create_Salesforceprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSF = create_Salesforceprogram("Strategic Analysis and Engagement")
Carbon_Free_BuildingsSF = create_Salesforceprogram("Carbon-Free Buildings")
CrossProgramImpactAccelerationSF = create_Salesforceprogram("Cross-Program Impact Acceleration")
Carbon_Free_ElectricitySF = create_Salesforceprogram("Carbon-Free Electricity")
UrbanTrans_SF = create_Salesforceprogram("Urban Transformation")
ClimateIntelSF = create_Salesforceprogram("Climate Intelligence")
IndustrySF = create_Salesforceprogram("Industry")
SEAsiaSF = create_Salesforceprogram("SE Asia Energy")
AfricaSF = create_Salesforceprogram("Africa")
USSF = create_Salesforceprogram("US")
BuildingElectrificationSF = create_Salesforceprogram("Building Electrification")
Energy_Transition_AcademySF = create_Salesforceprogram("Energy Transition Academy")
BeijingSF = create_Salesforceprogram("Beijing")
CFANSF = create_Salesforceprogram("CFAN")
D3SF = create_Salesforceprogram("Third Derivative")

Operations DF is EMPTY
Communications DF is EMPTY
Cross-Program Impact Acceleration DF is EMPTY
Building Electrification DF is EMPTY
Beijing DF is EMPTY
CFAN DF is EMPTY


In [226]:
Developing_Economies_RollupSF = pd.concat([IslandsSF, AfricaSF, SEAsiaSF, Energy_Transition_AcademySF, CFANSF], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSF = pd.concat([Carbon_Free_BuildingsSF, BuildingElectrificationSF], ignore_index = True, sort=False)
China_RollupSF = pd.concat([ChinaSF, BeijingSF], ignore_index = True, sort=False)

In [227]:
def create_MD_pivot5(Group):
    table = pd.pivot_table(Group, index = ['Opportunity Record Type','Initiative','Forecast Category','Opportunity Name', 'Documents Link', 'RMI Grant Name', 'Revenue Name','Stage', 'Revenue Date', 'Grant Term - Start Date', 'Grant Term - End Date', 'Grant Status', 'Program Fund Type', 'Probability (%)', 'Revenue Amount'], values = 'Expected Revenue', aggfunc=np.sum)
    return table

In [228]:
today = datetime.today()
timestamptoday = today.strftime("%B %d, %Y")

In [229]:
def to_excel_MD5(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="SF Data DEV&GovMLBL", startrow = 1)
    writer.save()
    writer.close()   

In [230]:
def edit_workbook_MD5(file):
    book = load_workbook(file)
    sheet = book["SF Data DEV&GovMLBL"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Opportunity Record Type"    
    sheet['B2']= "Initiative"
    sheet['C2']= "Forecast Category" 
    sheet['D2']= "Opportunity Name"
    sheet['E2']= "Essential Documents Link"      
    sheet['F2']= "RMI Grant Name"    
    sheet['G2']= "Revenue Name" 
    sheet['H2']= "Stage"     
    sheet['I2']= "Revenue Date"     
    sheet['J2']= "Grant Term - Start Date"
    sheet['K2']= "Grant Term - End Date"  
    sheet['L2']= "Grant Status"   
    sheet['M2']= "Program Fund Type"   
    sheet['N2']= "Probability (%)"     
    sheet['O2']= "Revenue Amount"
    sheet['P2']= "Expected Revenue"   
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True) 
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "SalesForce FY20+ Data as of " +timestamptoday
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 30
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 25
    sheet.column_dimensions['G'].width = 30
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['F'].hidden = True
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))
    for col in range(4, 5):
        for row in range(2, num_rows+1):
            if sheet.cell(column=5, row=row).value == 'na':
                continue
            else:
                try:
                    sheet.cell(column=col, row=row).hyperlink = sheet.cell(column=5, row=row).value  
                    sheet.cell(column=col, row=row).value = sheet.cell(column=col, row=row).value    
                    sheet.cell(column=col, row=row).style = "Hyperlink"   
                except:
                    pass    
    for col in range(4, 5):
        for row in range(2, num_rows+1):
                sheet.cell(column=col, row=row).border = thin_border
    for col in range(14, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 8):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Opportunity Record Type":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(8, 17):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 85  
    sheet.freeze_panes = sheet['H3']
    book.save(file)

In [231]:
def format_multi_pivot_MD5(Group, str):
    step1 = create_MD_pivot5(Group)
    to_excel_MD5(step1, MDpath+str)
    edit_workbook_MD5(MDpath+str)

In [232]:
format_multi_pivot_MD5(Developing_Economies_RollupSF, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(ClimateAligned_IndustriesSF, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(IndiaSF, '\\India - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD5(OperationsSF, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD5(CommunicationsSF, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(DevelopmentSF, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(China_RollupSF, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Breakthrough_Technology_SF, '\\M3 - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD5(CrossProgramImpactAccelerationSF, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(CarbonFree_MobilitySF, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Carbon_Free_ElectricitySF, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Strategic_Engagement_and_Analysis_GroupSF, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(CarbonFreeBuildings_RollupSF, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(UrbanTrans_SF, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(ClimateIntelSF, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(IndustrySF, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(USSF, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(GlobalClimate_FinanceSF, '\\Finance - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(BuildingElectrificationSF, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

## Create Salesforce Pivots for FFS

In [233]:
def create_Salesforceprogram2(program):
    firstdf = dfSalesforceFFS[dfSalesforceFFS['Program Area: Program Name']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [234]:
DevelopmentSFffs = create_Salesforceprogram2("General Support")
IslandsSFffs = create_Salesforceprogram2("Islands")
IndiaSFffs = create_Salesforceprogram2("India")
OperationsSFffs = create_Salesforceprogram2("Operations")
ChinaSFffs = create_Salesforceprogram2("China")
Breakthrough_Technology_SFffs = create_Salesforceprogram2("Breakthrough Technology")
CarbonFree_MobilitySFffs = create_Salesforceprogram2("Carbon-Free Mobility")
ClimateAligned_IndustriesSFffs = create_Salesforceprogram2("Climate Aligned Industries")
CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSFffs = create_Salesforceprogram2("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSFffs = create_Salesforceprogram2("Strategic Analysis and Engagement")
CrossProgramImpactAccelerationSFffs = create_Salesforceprogram2("Cross-Program Impact Acceleration")
Carbon_Free_BuildingsSFffs = create_Salesforceprogram2("Carbon-Free Buildings")
Carbon_Free_ElectricitySFffs = create_Salesforceprogram2("Carbon-Free Electricity")
UrbanTrans_SFffs = create_Salesforceprogram2("Urban Transformation")
ClimateIntelSFffs = create_Salesforceprogram2("Climate Intelligence")
IndustrySFffs = create_Salesforceprogram2("Industry")
SEAsiaSFffs = create_Salesforceprogram2("SE Asia Energy")
AfricaSFffs = create_Salesforceprogram2("Africa")
USSFffs = create_Salesforceprogram2("US")
BuildingElectrificationSFffs = create_Salesforceprogram2("Building Electrification")
Energy_Transition_AcademySFffs = create_Salesforceprogram2("Energy Transition Academy")
BeijingSFffs = create_Salesforceprogram2("Beijing")
CFANSFffs = create_Salesforceprogram2("CFAN")
D3SFffs = create_Salesforceprogram2("Third Derivative")

General Support DF is EMPTY
Operations DF is EMPTY
Communications DF is EMPTY
Strategic Analysis and Engagement DF is EMPTY
Cross-Program Impact Acceleration DF is EMPTY
Industry DF is EMPTY
Building Electrification DF is EMPTY
Beijing DF is EMPTY
CFAN DF is EMPTY


In [235]:
Developing_Economies_RollupSFffs = pd.concat([IslandsSFffs, AfricaSFffs, SEAsiaSFffs, Energy_Transition_AcademySFffs, CFANSFffs], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSFffs = pd.concat([Carbon_Free_BuildingsSFffs, BuildingElectrificationSFffs], ignore_index = True, sort=False)
China_RollupSFffs = pd.concat([ChinaSFffs, BeijingSFffs], ignore_index = True, sort=False)

In [236]:
def create_MD_pivot6(Group):
    table = pd.pivot_table(Group, index = ['Initiative','Account Name', 'Opportunity Name','Relationship Manager', 'Stage', 'Close Date', 'Probability (%)', 'Amount'], values = 'Weighted Value of Opportunity', aggfunc=np.sum)
    return table

In [237]:
def to_excel_MD6(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="SF Data FFS & Other Opps", startrow = 1)
    writer.save()
    writer.close()   

In [238]:
def edit_workbook_MD6(file):
    book = load_workbook(file)
    sheet = book["SF Data FFS & Other Opps"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Account Name"    
    sheet['C2']= "Opportunity Name"
    sheet['D2']= "Relationship Manager" 
    sheet['E2']= "Stage"
    sheet['F2']= "Close Date"    
    sheet['G2']= "Probability (%)"   
    sheet['H2']= "Amount"  
    sheet['I2']= "Weighted Value of Opportunity"   
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True) 
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:I1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['I1']
    top_left_cell.value = "SalesForce FY20+ Data as of " +timestamptoday 
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 35
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 25
    sheet.column_dimensions['H'].width = 25
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(7, 10):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 6):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,10):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(9, 10):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(6, 10):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 70  
    sheet.freeze_panes = sheet['F3']
    book.save(file)

In [239]:
def format_multi_pivot_MD6(Group, str):
    step1 = create_MD_pivot6(Group)
    to_excel_MD6(step1, MDpath+str)
    edit_workbook_MD6(MDpath+str)

In [240]:
format_multi_pivot_MD6(Developing_Economies_RollupSFffs, '\\Global South - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(ClimateAligned_IndustriesSFffs, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(IndiaSFffs, '\\India - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(OperationsSFffs, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(DevelopmentSFffs, '\\Development - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(CommunicationsSFffs, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(China_RollupSFffs, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Breakthrough_Technology_SFffs, '\\M3 - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(CrossProgramImpactAccelerationSFffs, '\\Cross-Program Impact Acceleration - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(CarbonFree_MobilitySFffs, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Carbon_Free_ElectricitySFffs, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(Strategic_Engagement_and_Analysis_GroupSFffs, '\\Strategic Analysis and Engagement - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(CarbonFreeBuildings_RollupSFffs, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(UrbanTrans_SFffs, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(ClimateIntelSFffs, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(IndustrySFffs, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(USSFffs, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(GlobalClimate_FinanceSFffs, '\\Finance - '+timestampMD+' - MD Package.xlsx')